In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,MaxPool2D 
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(42)
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import imageio 
import seaborn
import shutil
seaborn.set()

# Pretraining

In [ ]:
dataset = str(3000)
all_dict = "../input/tb" + dataset
pre_test_dict = "../input/tb500val"
pre_data_gen = ImageDataGenerator(rescale=1.0/255)

pre_train_gen = pre_data_gen.flow_from_directory(directory = all_dict, classes=["NORMAL", "TUBERCULOSIS"],class_mode="binary",target_size=(256, 256), shuffle = True)
pre_val_gen = pre_data_gen.flow_from_directory(directory = pre_test_dict, classes=["NORMAL", "TUBERCULOSIS"], class_mode="binary",target_size=(256, 256), shuffle = True)

In [ ]:
metrics = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name="auc"),

]

AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(256,256,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.5))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.5))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.5))

#Output Layer
AlexNet.add(Dense(1))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('sigmoid'))


In [ ]:
history = tf.keras.callbacks.History()
opt = keras.optimizers.Adadelta(learning_rate= 1e-3)
AlexNet.compile(optimizer=opt,loss='binary_crossentropy',metrics=metrics)
csv_logger = tf.keras.callbacks.CSVLogger('training_pretrained_TB' + dataset +".csv", append=True)
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(mintor="val_loss",min_lr=1e-6, factor=1e-2)



checkpoint_filepath = "./pretrained" + dataset + ".h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callbacks_list = [csv_logger,history]

In [ ]:
batch_size = 64

AlexNet.compile(optimizer=opt,loss='binary_crossentropy',metrics=metrics)
history = AlexNet.fit(x = pre_train_gen,epochs = 5, validation_data = pre_val_gen, batch_size = batch_size,callbacks = callbacks_list, shuffle=True)


In [ ]:
AlexNet.save( "alexnet_pretrainedTB" + dataset + ".h5")

# Finetunning****

In [ ]:
n = 30
model = "1000"


dataset = "dbcovid" + str(n)

train_dict = os.path.join("../input/",dataset)
test_dict = "../input/dbcovidval150"

data_gen = ImageDataGenerator(rescale=1.0/255)

train_gen = data_gen.flow_from_directory(directory = train_dict,classes = ["NORMAL", "COVID-19"], class_mode="binary",target_size=(256, 256))
val_gen = data_gen.flow_from_directory(directory = test_dict,classes = ["Normal", "COVID-19"], class_mode="binary", target_size=(256, 256))


In [ ]:
TL_file = tf.keras.models.load_model("./alexnet_pretrainedTB1000.h5")
TL_file.summary()

last_index = -15
    
last_conv_layer_output = TL_file.layers[last_index].output
dense1 = Dense(1000)(last_conv_layer_output)
BN1 = BatchNormalization(name="bn1")(dense1)
act1 = Activation('relu', name ="act1")(BN1)
dropout1 = Dropout(0.5)(act1)
dense2 = Dense(1)(dropout1)
BN2 = BatchNormalization(name="bn2")(dense2)
act2 = Activation('sigmoid', name ="act2" )(BN2)




TL = Model(TL_file.inputs, act2)

for i in TL.layers[:-7] :
    i.trainable = False

    
TL.summary()

In [ ]:


history_TL = tf.keras.callbacks.History()
history_baseline = tf.keras.callbacks.History()




reduceLR = tf.keras.callbacks.ReduceLROnPlateau(mintor="val_loss",min_lr=1e-6, factor=1e-2)
csv_logger_TL = tf.keras.callbacks.CSVLogger(dataset + model +' training_TL.csv', append=True)
csv_logger_baseline = tf.keras.callbacks.CSVLogger(dataset + model + ' training_baseline.csv', append=True)


callbacks_TL = [ csv_logger_TL ,history_TL ]
callbacks_baseline = [csv_logger_baseline,history_baseline ]




opt_TL = keras.optimizers.Adagrad(learning_rate=1e-3)
opt_BL = keras.optimizers.Adagrad(learning_rate=1e-3)

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr
    return lr

lr_metric_TL = get_lr_metric(opt_TL)
lr_metric_BL = get_lr_metric(opt_BL)

metrics_TL = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name="auc"),
        lr_metric_TL
    
]

metrics_BL = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name="auc"),
        lr_metric_BL
    
]



batch_size = 64

In [ ]:
TL.compile(optimizer=opt_TL,loss='binary_crossentropy',metrics=metrics_TL)


In [ ]:
TL_history = TL.fit(x = train_gen, validation_data = val_gen, epochs = 10 , batch_size = batch_size, callbacks= callbacks_TL)

In [ ]:
baseline = Sequential()

#1st Convolutional Layer
baseline.add(Conv2D(filters=96, input_shape=(256,256,3), kernel_size=(11,11), strides=(4,4), padding='same'))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))
baseline.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
baseline.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))
baseline.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
baseline.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))

#4th Convolutional Layer
baseline.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))

#5th Convolutional Layer
baseline.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))
baseline.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
baseline.add(Flatten())
# 1st Fully Connected Layer
baseline.add(Dense(4096, input_shape=(32,32,3,)))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))
# Add Dropout to prevent overfitting
baseline.add(Dropout(0.5))

#2nd Fully Connected Layer
baseline.add(Dense(4096))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))
#Add Dropout
baseline.add(Dropout(0.5))

#3rd Fully Connected Layer
baseline.add(Dense(1000))
baseline.add(BatchNormalization())
baseline.add(Activation('relu'))
#Add Dropout
baseline.add(Dropout(0.5))

#Output Layer
baseline.add(Dense(1))
baseline.add(BatchNormalization())
baseline.add(Activation('sigmoid'))


In [ ]:
baseline.compile(optimizer=opt_BL,loss='binary_crossentropy',metrics=metrics_BL)


In [ ]:
BL_history = baseline.fit(x = train_gen, validation_data = val_gen, epochs = 10 , batch_size = batch_size, callbacks= callbacks_baseline)


In [ ]:
plot_history = baseline_history

plt.plot(plot_history.history['accuracy'])
plt.plot(plot_history .history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plot_history = TL_history

plt.plot(TL_history.history['accuracy'])
plt.plot(TL_history .history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# non_pretrained_model = Sequential()
# non_pretrained_model.add(Conv2D(32, (3, 3), input_shape=(224,224,3)))
# non_pretrained_model.add(Activation('relu'))
# non_pretrained_model.add(MaxPooling2D(pool_size=(2, 2)))

# non_pretrained_model.add(Conv2D(32, (3, 3)))
# non_pretrained_model.add(Activation('relu'))
# non_pretrained_model.add(MaxPooling2D(pool_size=(2, 2)))

# non_pretrained_model.add(Conv2D(64, (3, 3)))
# non_pretrained_model.add(Activation('relu'))
# non_pretrained_model.add(MaxPooling2D(pool_size=(2, 2)))

# non_pretrained_model.add(Conv2D(64, (3, 3)))
# non_pretrained_model.add(Activation('relu'))
# non_pretrained_model.add(MaxPooling2D(pool_size=(2, 2)))



# non_pretrained_model.add(Flatten())
# non_pretrained_model.add(Dense(128))
# non_pretrained_model.add(Activation('relu'))
# non_pretrained_model.add(Dropout(0.5))
# non_pretrained_model.add(Dense(1))
# non_pretrained_model.add(Activation('sigmoid'))

In [ ]:
# opt2 = keras.optimizers.Adam(learning_rate=1e-2)

# non_pretrained_model.compile(optimizer=opt2,loss='binary_crossentropy',metrics=metrics)
# batch_size = 128
# non_pretrained_model.fit(x = train_gen, validation_data = val_gen,epochs = 50 , batch_size = batch_size, shuffle= True, callbacks = callbacks_scratch)
